In [1]:
import xarray as xr
import numpy as np
import os

In [2]:
def get_lonlat_idx_nearest_to_tgs(tg_ds,ds):
    '''tg_ds = xr.DataSet containing 'lon' and 'lat' coordinates of tide gauges
    ds    = xr.DataSet containing CMIP6 data to subset
    '''
    lon_name = list(k for k in ds.dims if 'lon' in k)[0] #find lon/lat coordinate names
    lat_name = list(k for k in ds.dims if 'lat' in k)[0]
    
    #compute distances between TG coordinates and grid cell centers
    distances = 2*np.arcsin( np.sqrt(
        np.sin( (np.pi/180) * 0.5*(ds[lat_name]-tg_ds.lat) )**2 +
        np.cos((np.pi/180)*tg_ds.lat)*np.cos((np.pi/180)*ds[lat_name])*np.sin((np.pi/180)*0.5*(ds[lon_name]-tg_ds.lon))**2) )

    distances = distances.where(np.isfinite(ds.sla.isel(time=0,drop=True)),1e30)
    idx_nearest = distances.argmin(dim=[lon_name,lat_name]) #find indices of nearest grid cells
    return idx_nearest

Regrid gridded satellite altimetry and ERA5-derived IBE to psmsl coordinates:

In [ ]:
psmsl = xr.open_dataset('../month_means_1993_2022_psmsl.nc')

In [11]:
#Regridding satellite altimetry and IBE at 1x1 degree grid:
sla = xr.open_dataset('../allsat_l4_duacs_sla_1993-2023_1x1.nc')
ibe = xr.open_dataset('../monthly_ibe_1993-2023_1x1.nc')

idx_nearest = get_lonlat_idx_nearest_to_tgs(psmsl,sla)

sla_at_tgs = sla.isel(lon=idx_nearest['lon'],lat=idx_nearest['lat'])
ibe_at_tgs = ibe.isel(lon=idx_nearest['lon'],lat=idx_nearest['lat'])

sla_at_tgs['lat'] = ('id',psmsl.lat.values)
sla_at_tgs['lon'] = ('id',psmsl.lon.values)

ibe_at_tgs['lat'] = ('id',psmsl.lat.values)
ibe_at_tgs['lon'] = ('id',psmsl.lon.values)

sla_at_tgs.to_netcdf('../allsat_l4_duacs_sla_1993-2023_at_psmsl.nc',mode='w') #store
ibe_at_tgs.to_netcdf('../monthly_ibe_1993-2023_at_psmsl.nc',mode='w')

In [3]:
#do the same for native satellite altimetry (more appropriate comparison?)
indir='../cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.125deg_P1M-m_202411/cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.125deg_P1M-m_202411'
fns = [os.path.join(indir,k) for k in os.listdir(indir) if k.startswith('.')==False]
ds = xr.open_mfdataset(fns)[['sla']]
ds = ds.rename({'latitude':'lat','longitude':'lon'})
ds = ds.load()
psmsl = xr.open_dataset('../month_means_1993_2022_psmsl.nc')


idx_nearest = get_lonlat_idx_nearest_to_tgs(psmsl,ds)
ds_at_tgs = ds.isel(lon=idx_nearest['lon'],lat=idx_nearest['lat'])

ds_at_tgs['lat'] = ('id',psmsl.lat.values)
ds_at_tgs['lon'] = ('id',psmsl.lon.values)
ds_at_tgs.to_netcdf('../duacs-0.125deg_P1M-m_202411_at_tgs.nc',mode='w') #store


/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:818: RuntimeWarning: invalid value encountered in arcsin
  result_data = func(*input_data)
